In [ ]:
from typing import Optional

from scipy import integrate
from scipy.linalg import expm
import numpy as np
from matplotlib import pyplot as plt

from control_theory import systems, utils

In [ ]:
pend = systems.PendODESystem()

In [ ]:
h = 0.1

A_d = expm(pend.A * h)

print(A_d)

In [ ]:
def f(x):
    return expm(pend.A * x) @ pend.b

In [ ]:
b_d = integrate.quad_vec(f, 0, h)[0]

print(b_d)

In [ ]:
C = np.column_stack([
    b_d,
    A_d @ b_d,
    np.linalg.matrix_power(A_d, 2) @ b_d,
    np.linalg.matrix_power(A_d, 3) @ b_d
])

print(C)
print(C.shape)

In [ ]:
print(np.linalg.matrix_rank(C))

In [ ]:
print(np.linalg.eigvals(C))

In [ ]:
theta = (
    -np.array([[0, 0, 0, 1]])
    @ np.linalg.inv(C)
    @ utils.create_char_pol(0.7, -0.9, 0.9, 0.7)(A_d)
)

In [ ]:
start, stop = 0, 10


time = np.linspace(start, stop, 300)
y_0 = np.array([0, 0.1, 0, 0])

sol = integrate.solve_ivp(
    systems.linear_system,
    (start, stop),
    y_0,
    dense_output=True,
    args=(pend.A, pend.b, theta),
    method="RK45"
)

In [ ]:
z = sol.sol(time)

In [ ]:
y_labels = (r"x", r"\phi", r"\dot x", r"\dot \phi")
# plt.rcParams['text.usetex'] = True # uncomment if you have latex

fig, axs = plt.subplots(4, 1)
fig.set_size_inches(10, 15)

for i in range(4):
    axs[i].plot(time, z[i])
    axs[i].set_xlabel('time')
    axs[i].set_ylabel(y_labels[i])
    axs[i].grid(True)

fig.tight_layout()
# fig.savefig('discrete.png', dpi=300, facecolor='white') # uncomment to save high-res picture
plt.show()

In [ ]:
start, stop = 0, 10
n = 100
eps = 0.01

# Согласовываем шаг интегрирования и шаг с которым брали интеграл для получения b_d
# Ну то есть проверяем))0
assert h - eps <= (stop - start) / n <= h + eps

time = np.linspace(start, stop, n)
y_0 = np.array([0, 0.1, 0, 0])

xs = []

for i in range(len(time) - 1):
    begin, end = time[i], time[i + 1]
    sol = integrate.solve_ivp(systems.linear_system, (begin, end), y_0, args=(pend.A, pend.b, theta), method="RK45", t_eval=[end])
    xs.append(sol.y.ravel())
    y_0 = sol.y.ravel()

xs = np.asarray(xs)

In [ ]:
y_labels = (r"x", r"\phi", r"\dot x", r"\dot \phi")
# plt.rcParams['text.usetex'] = True # uncomment if you have latex

fig, axs = plt.subplots(4, 1)
fig.set_size_inches(10, 15)

for i in range(4):
    axs[i].plot(time[:-1], xs[:, i])
    axs[i].set_xlabel('time')
    axs[i].set_ylabel(y_labels[i])
    axs[i].grid(True)

fig.tight_layout()
# fig.savefig('out_discrete_2.png', dpi=300, facecolor='white') # uncomment to save high-res picture
plt.show()

## Вариант в

In [ ]:
def f_c(x):
    return expm(pend.A * (0 + delta - x)) @ pend.b

In [ ]:
delta = 0.5 * h

b_d = integrate.quad_vec(f_c, 0, h)[0]

print(b_d)

In [ ]:
C = np.column_stack([
    b_d,
    A_d @ b_d,
    np.linalg.matrix_power(A_d, 2) @ b_d,
    np.linalg.matrix_power(A_d, 3) @ b_d
])

print(C)
print(C.shape)

In [ ]:
print(np.linalg.matrix_rank(C))
print(np.linalg.eigvals(C))

In [ ]:
theta = (
    -np.array([[0, 0, 0, 1]])
    @ np.linalg.inv(C)
    @ utils.create_char_pol(0.7, -0.9, 0.9, 0.7)(A_d)
)

print(theta)

In [ ]:
def system(
        t: np.ndarray,
        x: np.ndarray,
        A: np.ndarray,
        b: np.ndarray,
        theta: np.ndarray,
        delta: float,
        h: float,
        x_0: Optional[np.ndarray] = None,
) -> np.ndarray | int:
    if x_0 is None:
        x_0 = x

    if t + delta < t + h:
        return A @ x + b @ theta @ x_0

    return A @ x

In [ ]:
start, stop = 0, 10
n = 200
eps = 0.01

# Согласовываем шаг интегрирования и шаг с которым брали интеграл для получения b_d
# Ну то есть проверяем))0
assert delta - eps <= (stop - start) / n <= delta + eps

time = np.linspace(start, stop, n)
y_0 = np.array([0, 0.1, 0, 0])

xs = []

for i in range(len(time) - 1):
    begin, end = time[i], time[i + 1]

    sol = integrate.solve_ivp(
        system,
        (begin, end), y_0,
        args=(pend.A, pend.b, theta, delta, h),
        method="RK45",
        t_eval=[end]
    )

    xs.append(sol.y.ravel())
    y_0 = sol.y.ravel()

xs = np.asarray(xs)

In [ ]:
y_labels = (r"x", r"\phi", r"\dot x", r"\dot \phi")
# plt.rcParams['text.usetex'] = True # uncomment if you have latex

fig, axs = plt.subplots(4, 1)
fig.set_size_inches(10, 15)

for i in range(4):
    axs[i].plot(time[:-1], xs[:, i])
    axs[i].set_xlabel('time')
    axs[i].set_ylabel(y_labels[i])
    axs[i].grid(True)

fig.tight_layout()
# fig.savefig('out_discrete_2.png', dpi=300, facecolor='white') # uncomment to save high-res picture
plt.show()